# music system based on facial expressions

In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all training images
train_generator = train_data_gen.flow_from_directory(
    'data/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

# Preprocess all validation images
validation_generator = validation_data_gen.flow_from_directory(
    'data/test',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

# Create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

# Train the neural network/model
emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=200,  # Adjust the number of steps per epoch as needed
    epochs=5,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=5  # Adjust the number of validation steps as needed
)

# Save the trained model
emotion_model.save('emotion_model.h5')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


C:\Users\rawat\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\rawat\AppData\Local\Temp/ipykernel_1800/99326359.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model.fit_generator(


Epoch 1/5
200/200 [==============================] - 204s 1s/step - loss: 1.8286 - accuracy: 0.2428 - val_loss: 1.8338 - val_accuracy: 0.2500
Epoch 2/5
200/200 [==============================] - 203s 1s/step - loss: 1.7932 - accuracy: 0.2616 - val_loss: 1.7856 - val_accuracy: 0.2719
Epoch 3/5
200/200 [==============================] - 205s 1s/step - loss: 1.6993 - accuracy: 0.3266 - val_loss: 1.7028 - val_accuracy: 0.3719
Epoch 4/5
200/200 [==============================] - 139s 692ms/step - loss: 1.6098 - accuracy: 0.3770 - val_loss: 1.4891 - val_accuracy: 0.4531
Epoch 5/5
200/200 [==============================] - 123s 614ms/step - loss: 1.5587 - accuracy: 0.4011 - val_loss: 1.5010 - val_accuracy: 0.4500


In [2]:
import cv2
from keras.models import load_model
import numpy as np
import random
import pygame
import time
from IPython.display import display, clear_output

# Load the pre-trained CNN model for facial expression recognition
emotion_model = load_model('emotion_model.h5')

# Define the list of emotion labels
emotion_labels = ['Angry', 'Disgusted', 'Fearful', 'Happy', 'Neutral', 'Sad', 'Surprised']

# Start the webcam feed
cap = cv2.VideoCapture(0)
cv2.namedWindow('Emotion Detection')

# Variables to store emotion predictions
emotion_predictions = []

# Set the duration for capturing frames (in seconds)
duration = 10

# Set the start time
start_time = time.time()

# Initialize pygame mixer
pygame.mixer.init()

# Mood-based song recommendations
mood_songs = {
    "happy": ["Luke-Bergs-Bliss.mp3","Feel-Good.mp3"],
    "sad": ["sad1.mp3","sad2.mp3"],
    "Neutral": ["neutral1.mp3",]
    # Add more mood-song mappings as needed
}

# Set the prediction frequency (process every nth frame)
prediction_frequency = 5
frame_counter = 0

# Flag to indicate if the song is currently playing
song_playing = False

while time.time() - start_time < duration:

    ret, frame = cap.read()
    if not ret:
        break
    
    frame_counter += 1

    # Process every nth frame
    if frame_counter % prediction_frequency == 0:
        # Preprocess the frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (48, 48))
        gray = np.reshape(gray, (1, 48, 48, 1))
        gray = gray.astype('float32') / 255

        # Make a prediction on the preprocessed frame
        emotion_prediction = emotion_model.predict(gray)
        emotion_index = np.argmax(emotion_prediction)
        emotion_label = emotion_labels[emotion_index]
        
        # Append the predicted emotion to the list
        emotion_predictions.append(emotion_label)

        # Display the emotion label on the frame
        cv2.putText(frame, emotion_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
     
    # Display the frame
    cv2.imshow('Emotion Detection', frame)
    cv2.waitKey(1)
    
    # Check if the 's' key is pressed to stop the song
    if cv2.waitKey(1) & 0xFF == ord('s') and song_playing:
        # Stop the song using pygame mixer
        pygame.mixer.music.stop()
        song_playing = False
        print("Song stopped.")
        # Clear the output to remove the previous displayed frame
        clear_output(wait=True)
        display(frame)
        
    # Check if the 's' key is pressed to save the frame
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # Save the frame as an image
        cv2.imwrite('saved_frame.jpg', frame)
        print("Frame saved as saved_frame.jpg.")
        
# Determine the final emotion based on the majority vote
final_emotion = max(set(emotion_predictions), key=emotion_predictions.count)

# Print the final emotion
print("Final Emotion:", final_emotion)

# Check if songs are available for the final emotion
songs = mood_songs.get(final_emotion.lower())
if songs:
    # Select a random song from the available songs
    song = random.choice(songs)
    
    # Load and play the song using pygame mixer
    pygame.mixer.music.load(song)
    pygame.mixer.music.play()
    song_playing = True
    print(f"Now playing: {song}")
else:
    print("Sorry, no songs found for the specified mood.")

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 41ms/step
Final Emotion: Sad
Now playing: sad1.mp3
